# Practical PyTorch: Classify Authors with a sentence-Level RNN

In [2]:
!pip install tensorflow

  Using cached tensorflow-2.13.1-cp38-cp38-macosx_12_0_arm64.whl.metadata (2.6 kB)
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Using cached tensorflow-2.13.0-cp38-cp38-macosx_12_0_arm64.whl.metadata (2.6 kB)
  Using cached tensorflow_macos-2.13.0-cp38-cp38-macosx_12_0_arm64.whl.metadata (3.2 kB)
  Using cached absl_py-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.10.0-cp38-cp38-macosx_11_0_arm64.whl.metadata (2.5 kB)
  Using cached libclang-16.0.6-py2.py3-none-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached numpy-1.24.3-cp38-cp38-macosx_11_0_arm64.whl (13.8 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 k

### Imports

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

### Read Input (.csv with Author, Sentence)

In [10]:
data = pd.read_csv('grand_sentences.csv', encoding='latin-1')
data['Sentence'] = data['Sentence'].replace(np.nan, '')
sentences = data['Sentence'].values
authors = data['Author'].values

### Tokenize

In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1


sequences = tokenizer.texts_to_sequences(sentences)
max_sequence_len = max([len(seq) for seq in sequences])

### Pad sequences

In [14]:
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post')

### Encode Author Names

In [21]:
unique_authors = list(set(authors))
author_to_label = {author: idx for idx, author in enumerate(unique_authors)}
labels = np.array([author_to_label[author] for author in authors])  # Convert to numpy array
len(labels)

16887

### Split data into training and testing sets

In [22]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

### Build an RNN model

In [23]:
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_len))
model.add(LSTM(units=128))
model.add(Dense(units=len(unique_authors), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Fit the model

In [24]:
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10
212/212 [==============================] - 483s 2s/step - loss: 2.0214 - accuracy: 0.1914 - val_loss: 2.0169 - val_accuracy: 0.1915
Epoch 2/10
212/212 [==============================] - 469s 2s/step - loss: 2.0181 - accuracy: 0.1910 - val_loss: 2.0167 - val_accuracy: 0.1915
Epoch 3/10
212/212 [==============================] - 471s 2s/step - loss: 2.0176 - accuracy: 0.1957 - val_loss: 2.0163 - val_accuracy: 0.1951
Epoch 4/10
212/212 [==============================] - 498s 2s/step - loss: 2.0176 - accuracy: 0.1947 - val_loss: 2.0176 - val_accuracy: 0.1951
Epoch 5/10
212/212 [==============================] - 704s 3s/step - loss: 2.0170 - accuracy: 0.1950 - val_loss: 2.0152 - val_accuracy: 0.1915
Epoch 6/10
212/212 [==============================] - 1039s 5s/step - loss: 2.0170 - accuracy: 0.1945 - val_loss: 2.0162 - val_accuracy: 0.1915
Epoch 7/10
212/212 [==============================] - 461s 2s/step - loss: 2.0171 - accuracy: 0.1962 - val_loss: 2.0150 - val_accuracy: 0.191

### Evaluate the model

In [25]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

106/106 [==============================] - 53s 503ms/step - loss: 2.0144 - accuracy: 0.1915
Test Accuracy: 19.15%
